# Visualize geometry

In [1]:
#check the contents of .inp file
refer = "inp_files/Demi-B10L_R00_n01.inp"
fopen = open(refer, "r")
line = fopen.readlines()

line[:20]

['*Heading\n',
 '** Generated with Matlab script\n',
 '*Preprint, echo=NO, model=NO, history=NO, contact=NO\n',
 '**\n',
 '** PARTS\n',
 '**\n',
 '*Part, name=Lattice\n',
 '*Node\n',
 '      1,      0.000000,      8.660254\n',
 '      2,      0.000000,     35.980763\n',
 '      3,      0.000000,     63.301272\n',
 '      4,      0.000000,     90.621781\n',
 '      5,      3.660254,     22.320509\n',
 '      6,      3.660254,     49.641018\n',
 '      7,      3.660254,     76.961527\n',
 '      8,      3.660254,    104.282036\n',
 '      9,      8.660254,      3.660254\n',
 '     10,      8.660254,     13.660255\n',
 '     11,      8.660254,     30.980761\n',
 '     12,      8.660254,     40.980764\n']

In [2]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

input_folder = "inp_files"
output_folder = "data_viz"

#if visualization for all inp files
#inp_files = [f for f in os.listdir(input_folder) if f.endswith(".inp")]
#output_folder = "plots"

preflix = 'Demi-B10L_'

files = ['R00_n01.inp',
        'R01_n01.inp', 'R01_n02.inp', 'R01_n03.inp', 'R01_n10.inp', 'R01_n20.inp',
        'R20_n01.inp', 'R20_n02.inp', 'R20_n03.inp', 'R20_n10.inp', 'R20_n20.inp',
        'R50_n01.inp', 'R50_n02.inp', 'R50_n03.inp', 'R50_n10.inp', 'R50_n20.inp']

inp_files = [preflix + f for f in files]

# Move these to the top of your script (outside the loop)
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['font.size'] = 13

#extract edges and nodes from each file
for file_name in inp_files:
    file = os.path.join(input_folder, file_name)
    data = open(file, "r")
    lines = data.readlines()
    
    nodes = []
    elements = []
    recording_nodes = False
    recording_elements = False
    
    for line in lines:
        line = line.strip()
        if line.startswith("*Node"):
            recording_nodes = True
            continue
        elif line.startswith("*Element"):
            recording_nodes = False
            recording_elements = True
            continue
        elif line.startswith("*") and recording_elements:
            break

        if recording_nodes:
            parts = line.split(",")
            node_id = int(parts[0])
            x, y = float(parts[1]), float(parts[2])
            nodes.append((node_id, x, y))
        
        if recording_elements:
            parts = line.split(",")
            element_id = int(parts[0])
            node1, node2 = int(parts[1]), int(parts[2])
            elements.append((element_id, node1, node2))
    
    node_dict = {node[0]: (node[1], node[2]) for node in nodes}
    nodes_array = np.array(nodes)

    #plot each lattice
    fig, ax = plt.subplots(figsize=(5, 5), dpi=300)

    #plot edges
    for _, n1, n2 in elements:
        x_values = [node_dict[n1][0], node_dict[n2][0]]
        y_values = [node_dict[n1][1], node_dict[n2][1]]
        ax.plot(x_values, y_values, color='blue', lw=1.4, alpha=0.6)

    #plot nodes
    ax.scatter(nodes_array[:, 1], nodes_array[:, 2], s=24, color='black', zorder=2)

    name = os.path.splitext(file_name)[0]
    ax.set_title(f"{name}")
    ax.set_aspect('equal')
    save_path = os.path.join(output_folder, f"{name}.png")
    fig.tight_layout()
    fig.savefig(save_path)
    plt.close(fig) 
